# This script obtains augmented data out of cmu skeletons from different cameras
Input: ../170407_office2/samples_for_persons.json
Output: /170407_office2/hd_00_samples_for_persons.json
        /170407_office2/hd_01_samples_for_persons.json
        ...
        /170407_office2/hd_30_samples_for_persons.json
p.s. The structure of hd_00_samples_for_persons.json is still the same as that of samples_for_persons.json, but the x1,y1,z1 changed to the calibrated version with respect to each camera

In [1]:
import numpy as np
import json
import panutils
from IPython.core.debugger import set_trace
import pandas as pd
from collections import OrderedDict

In [2]:
# Setup paths
data_path = '../'
# seq_name = '170915_office1'#5376
seq_name = '170407_office2' #3649+1211=4254

In [3]:
# Load camera calibration parameters
with open(data_path+seq_name+'/calibration_{0}.json'.format(seq_name)) as cfile:
    calib = json.load(cfile)
# Cameras are identified by a tuple of (panel#,node#)
# Select an HD camera (0,0) - (0,30), where the zero in the first index means HD camera 
cameras = {(cam['panel'],cam['node']):cam for cam in calib['cameras'] if cam['panel']==0}

In [4]:
# Read the skeleton data from ../170407_office2/samples_for_persons.json
try:
    with open (data_path+seq_name+"/samples_for_persons.json") as skeleton_file:
        skeleton=json.load(skeleton_file)
    set_trace()
except IOError as e:
    print('Error reading {0}\n'.format(data_path+seq_name+"/samples_for_persons.json")+e.strerror)  

--Return--
None
> <ipython-input-4-3d366360f568>(5)<module>()
      3     with open (data_path+seq_name+"/samples_for_persons.json") as skeleton_file:
      4         skeleton=json.load(skeleton_file)
----> 5     set_trace()
      6 except IOError as e:
      7     print('Error reading {0}\n'.format(data_path+seq_name+"/samples_for_persons.json")+e.strerror)

ipdb> c


In [5]:
# create the samples_for_persons of each camera
# the output is: /170407_office2/hd_00_samples_for_persons.json
for k,cam in cameras.items():
    # Convert data into numpy arrays for convenience
    cam['K'] = np.matrix(cam['K'])
    cam['distCoef'] = np.array(cam['distCoef'])
    cam['R'] = np.matrix(cam['R'])
    cam['t'] = np.array(cam['t']).reshape((3,1))
    new_skeleton=dict()
    for person, frames in skeleton.items():
        # There are 19 3D joints for each frame, stored as an array [x1,y1,z1,c1,x2,y2,z2,c2,...]
        # frames is the stack of [x1,y1,z1,c1,x2,y2,z2,c2,...] over all the frames,
        # frames have the shape: (# of frames)*76 
        # where c1 ... c19 are per-joint detection confidences
#         new_frames=list()
        new_frames=OrderedDict()
        sorted_frame_keys=sorted([int(k) for k in frames.keys()])
        for frmidx in sorted_frame_keys: 
#             print(frmidx)
            frame=np.array(frames[str(frmidx)])#frame.shape: 1*76
            skel = frame.reshape(-1,4).transpose()# skel shape: 4*19

            # Project skeleton into view (this is like cv2.projectPoints)
            pt = panutils.projectPoints(skel[0:3,:],
                          cam['K'], cam['R'], cam['t'], 
                          cam['distCoef'])
            skel[0:3]=pt
            skel=list(skel.transpose().reshape(-1))
#             new_frames.append(skel)
            new_frames[frmidx]=skel
        # after the loop, the new_frames will be in the same shape as frames
        new_skeleton[person]=new_frames
#     set_trace()
    #after building the new skeleton, write the json file with respect to the camemra
    with open(data_path+seq_name+"/camerawise_skeleton"+"/hd_"+str(k[0])+'_'+str(k[1])+"_samples_for_persons.json","w+") as f:
        json.dump(new_skeleton,f)
    
        
            
           
            